# Regresión múltiple

- Tarea: entrenar un modelo que predice el Precio de un vehículo, basado en su kilometraje, número de cilindros y año de fabricación

- Dataset: https://www.kaggle.com/austinreese/craigslist-carstrucks-data/version/10

## Preparación de los datos

Carguemos nuestros dataset de carros usados con precios y otras caracteristicas

Visualizamos los vehiculos agrupados por rangos de kilometraje

Podemos usar pandas para seleccionar solamente las caracteristicas en las que estamos interesados, y el valor que estamos tratando de predecir

En este ejemplo estamos evitando datos como la marca (manufacturer) o el modelo (model) ya que las regresiones no funcionan bien con este tipo de datos, al menos que puedas convertirlos a algún valor númerico.

En este caso, usaremos pandas para convertir el valor de número de cilindros (cyllinders) a entero.

## Entrenando mi modelo

Creamos dos arrays:

- array de características, lo llamamos X (por convención se usa X en mayúsculas)

- array de labels (valores a predecir), lo llamamos y (por convención se usa y en minúsculas)

Escalemos nuestras características en el mismo rango para que podamos comparar fácilmente los coeficientes con los que terminamos.

Agregamos una columna constante a nuestro modelo de tal manera que podamos tener una intercepción

Entrenamos nuestro modelo de regresión OLS (ordinary least squares)

La tabla de coeficientes anterior nos da los valores para insertar en una ecuación de forma:
    B0 + B1 * odometer + B2 * cylinders + B3 * year
    
En este ejemplo, está bastante claro que la cantidad de cilindros y el año de fabricación son más importantes basado en los coeficientes.

## Usando mi modelo para realizar predicciones

Comencemos escalando nuestras características en la misma escala que usamos para entrenar el modelo

Llame la función est.predict(...) pasando las características escaladas